In [14]:
import numpy as np
import matplotlib.pyplot as plt
import dwave_networkx as dnx
import networkx as nx
%matplotlib inline
import matplotlib as mpl
import math
from dwave.system import DWaveSampler
import minorminer as mm
import dwave.inspector

In [10]:
# Use a D-Wave system as the sampler
f = open("token.txt", "r")
#sampler = DWaveSampler(solver=dict(topology__type='chimera'),token=f.read())
sampler = DWaveSampler(solver=dict(topology__type='zephyr'),token=f.read())

print("QPU {} was selected.".format(sampler.solver.name))

QPU Advantage2_prototype1.1 was selected.


In [7]:
print("Chosen topology is:")
print(sampler.properties["topology"])
print("Maximum anneal-schedule points: {}".format(sampler.properties["max_anneal_schedule_points"]))
print("Annealing time range: {} (in us)".format(sampler.properties["annealing_time_range"]))

max_slope = 1.0/sampler.properties["annealing_time_range"][0]

print("Maximum slope allowed on this solver is {:.2f}.".format(max_slope))
print("J range: {}".format(sampler.properties["j_range"]))
print("h range: {}".format(sampler.properties["h_range"]))
print("Time range of problems sent to QPU: {} s".format( (1.0 /10**6) * np.array(sampler.properties["problem_run_duration_range"])))
   

Chosen topology is:
{'type': 'zephyr', 'shape': [4, 4]}
Maximum anneal-schedule points: 12
Annealing time range: [1.0, 2000.0] (in us)
Maximum slope allowed on this solver is 1.00.
J range: [-1.0, 1.0]
h range: [-4.0, 4.0]
Time range of problems sent to QPU: [0. 1.] s


# test spin reversal transform

In [15]:
from dwave.system import DWaveSampler, FixedEmbeddingComposite
f = open("token.txt", "r")

qpu = DWaveSampler(solver=dict(topology__type='zephyr'),token=f.read())
Q = {('x1', 'x2'): 1, ('x1', 'z'): -2, ('x2', 'z'): -2, ('z', 'z'): 3}
       


In [17]:
#create a target graph
T=nx.Graph()
T.add_edges_from(sampler.edgelist)
T.add_nodes_from(sampler.nodelist)

#create a source graph
S=nx.Graph()
S.add_node(1)
S.add_node(2)
S.add_node(3)


S.add_edge(1,2)
S.add_edge(2,3)
S.add_edge(3,1)

embedding=mm.find_embedding(S,T)

In [18]:
embedding

{1: [425], 2: [82], 3: [428]}

In [31]:

Q = {('1', '2'): 1, ('1', '3'): -2, ('2', '3'): -2, ('3', '3'): 3}
#need to transfer to Ising medel !!!!

couplers = {(425, 82): 1/4, (425, 428): -1/2, (82, 428): -1/2}
hdict = {425:-1/4, 82:-1/4, 428:1/2}


In [28]:
embedding

{1: [425], 2: [82], 3: [428]}

In [30]:
qpu

In [38]:
#1sampleset = FixedEmbeddingComposite(qpu, embedding).sample_qubo(Q, num_reads=500)   
num_reads = 500
sampleset = qpu.sample_ising(h=hdict,
        J=couplers,
        num_reads=num_reads)
print("Feasible: {} percent. Time: {} us".format(
      sampleset.record.num_occurrences[:4]/num_reads,
      sampleset.info["timing"]["qpu_access_time"]))  

Feasible: [0.21  0.254 0.214 0.322] percent. Time: 39395.21 us


In [41]:
#1sampleset = FixedEmbeddingComposite(qpu, embedding).sample_qubo(Q, num_reads=500)   
num_reads = 500
sampleset = qpu.sample_ising(h=hdict,
        J=couplers,
        num_reads=num_reads,
        num_spin_reversal_transforms=10)
print(sampleset.record)


[([-1, -1, -1], -0.75, 136) ([-1,  1, -1], -0.75, 102)
 ([ 1, -1, -1], -0.75, 123) ([ 1,  1,  1], -0.75, 139)]


In [42]:
print("Feasible: {} percent. Time: {} us".format(
      sampleset.record.num_occurrences[:4]/num_reads,
      sampleset.info["timing"]["qpu_access_time"]))

Feasible: [0.272 0.204 0.246 0.278] percent. Time: 101002.09999999998 us
